In [1]:
# Use huggingface datasets
from datasets import Dataset
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import transformers
import mlflow
import importlib
import json

import sys
sys.path.append("../")
from src import models

importlib.reload(models)
importlib.reload(models.transformer_models)
importlib.reload(models.train_model)
importlib.reload(models.eval_model)

<module 'src.models.eval_model' from '/home/mas-server/etu/nn/paraphrase_detection/notebooks/../src/models/eval_model.py'>

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("transformers")

<Experiment: artifact_location='/home/mas-server/etu/nn/paraphrase_detection/notebooks/mlruns/2', creation_time=1711215132962, experiment_id='2', last_update_time=1711215132962, lifecycle_stage='active', name='transformers', tags={}>

In [3]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

Using CUDA


In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained("microsoft/deberta-v3-large", use_fast=True)
def tokenize_function(samples):
    return tokenizer(            
        samples["question1"], # first string
        samples["question1"], # second string
        return_tensors="pt", # return torch tensor
        padding="max_length", # Pad seqeunces
        max_length=128, # Max len for padded seq
        truncation=True, # Truncate string
        return_token_type_ids=True, # Return mask for q1 and q2
    )
def collate_fn(data):
    input_ids = torch.stack([example["input_ids"] for example in data])
    labels = torch.stack([example["labels"] for example in data]).reshape(-1, 1)
    masks = torch.stack([example["attention_mask"] for example in data]).bool()
    return {"input_ids": input_ids, "labels": labels, "attention_mask" : masks}

/home/mas-server/anaconda3/envs/ocean/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
data_dir = '../data/processed'
TRAIN = 'train'
TEST = 'test'
VAL = 'val'
text_datasets = {
    x : Dataset.from_pandas(pd.read_csv(data_dir + "/" + x + ".csv"))
        # Tokenize questions
        .map(tokenize_function, batched = True)
        # Rename is_duplicate to labels
        .map(lambda examples: {"labels": examples["is_duplicate"]}, batched=True
        )
    for x in [TRAIN, VAL, TEST]
}
for dataset in text_datasets:
    text_datasets[dataset].set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])
dataloaders = {
    x: torch.utils.data.DataLoader(
        text_datasets[x], batch_size = 128,
        shuffle=True, collate_fn = collate_fn
    )
    for x in [TRAIN, VAL, TEST]
}

Map:   0%|          | 0/283000 [00:00<?, ? examples/s]

Map:   0%|          | 0/283000 [00:00<?, ? examples/s]

Map:   0%|          | 0/60643 [00:00<?, ? examples/s]

Map:   0%|          | 0/60643 [00:00<?, ? examples/s]

Map:   0%|          | 0/60644 [00:00<?, ? examples/s]

Map:   0%|          | 0/60644 [00:00<?, ? examples/s]

In [6]:
def run_mlflow_experiment(
    exp_name,
    model_config_path,
    dataloaders,
    epochs
):
    with open(model_config_path, "r") as f:
        model_config = json.load(f)
    model_type = model_config.get("type", "No type in config")
    if model_type == "lstm":
        model = models.lstm_models.build_SimpleBiLSTM(model_config)
    elif model_type == "residual_lstm":
        raise NotImplemented()
    elif model_type == "transformer":
        model = models.transformer_models.build_transfomer(model_config)
    with mlflow.start_run(run_name=exp_name):
        print(f"Model config: {model_config}")
        model.to("cuda")
        mlflow.log_params(model_config)
        mlflow.log_param("epochs", epochs)
        mlflow.set_tag("model_name", model_type)
        model = models.train_model.train_model(
            model,
            torch.nn.BCEWithLogitsLoss(),
            optim.Adam(model.parameters(), weight_decay=1e-4),
            dataloaders,
            model_type,
            epochs
        )
        models.eval_model.eval_model(model, dataloaders, model_type)
        mlflow.pytorch.log_model(model, "torch_models")

In [7]:
config_paths = "../models/"
configs = [x for x in os.listdir(config_paths) if x.endswith(".json") and x.startswith("transformer")]
print(configs)

['transformer0.json']


In [8]:
import os 
# Disable tokenizers parallel to disable nasty warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"
for conf in configs:
    run_mlflow_experiment(
        conf[:-5],
        config_paths + conf,
        dataloaders,
        50
    )

Model config: {'vocab_size': 128000, 'embedding_dim': 80, 'maxlen': 128, 'nhead': 4, 'd_hid': 150, 'dropout': 0.5}
Epoch 0/50
----------
Training batch 2210/2211
Validation batch 473/474
Epoch 1/50
----------
Training batch 2210/2211
Validation batch 473/474
Epoch 2/50
----------
Training batch 2210/2211
Validation batch 473/474
Epoch 3/50
----------
Training batch 2210/2211
Validation batch 473/474
Epoch 4/50
----------
Training batch 2210/2211
Validation batch 473/474
Epoch 5/50
----------
Training batch 2210/2211
Validation batch 473/474
Epoch 6/50
----------
Training batch 2210/2211
Validation batch 473/474
Epoch 7/50
----------
Training batch 2210/2211
Validation batch 473/474
Epoch 8/50
----------
Training batch 2210/2211
Validation batch 473/474
Epoch 9/50
----------
Training batch 2210/2211
Validation batch 473/474
Epoch 10/50
----------
Training batch 2210/2211
Validation batch 473/474
Epoch 11/50
----------
Training batch 2210/2211
Validation batch 473/474
Epoch 12/50
-------